# Generación de texto con LSTMs

In [1]:
# Paso 1: importar librerías
import re
import numpy as np
import pandas as pd
from sklearn import preprocessing
from keras import models, layers

Using TensorFlow backend.


In [2]:
# load ascii text, covert to lowercase and delete special characters
f = open('MobyDick.txt', 'r', encoding="utf8") 
text = f.read()
f.close()
text = text.lower()
text = re.sub(r"[^a-zA-Z0-9]+", ' ', text)

# create mapping of unique chars to integers, and a reverse mapping
chars = np.array(list(text))
encoder = preprocessing.LabelEncoder().fit(chars)
encoded_chars = encoder.transform(chars)

# normalize data
'''
mean = encoded_chars.mean()
stdv = encoded_chars.std()
encoded_chars = (encoded_chars - mean) / stdv
'''
unique_count = len(np.unique(encoded_chars))

In [3]:
#paso 3: preparar datos: 
# 1- preprocesar: ejemplo: CHAPT -> E; HAPTE -> R (pero con 100 caracteres)

# split a univariate sequence into samples
def split_sequence(sequence, n_steps, n_step_diff=1):
    X, y = list(), list()
    i = 0
    while i <len(sequence): 
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        i = i + (n_step_diff)
    return np.array(X), np.array(y)

X_train, y_train = split_sequence(encoded_chars, 100, 100)

# 2- cambiar dataset para que la entrada sea válida: [Samples, time_steps,features]
n_samples = X_train.shape[0]
n_steps = X_train.shape[1] # 100
n_features = 1
X_train = X_train.reshape((n_samples, n_steps, n_features))

def int_to_array(y_train, length):
    y_list = []
    for i in y_train:        
        a = np.zeros(length, dtype=float)
        a[i] = 1
        y_list.append(a)
    return np.array(y_list)

y_train = int_to_array(y_train, unique_count)

37

In [4]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
# definimos modelo LSTM
lstm_model = models.Sequential([
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.5, return_sequences=True, input_shape=X_train.shape[-2:]),
    layers.LSTM(32, dropout=0.1, recurrent_dropout=0.5),
    layers.Dense(unique_count, activation='softmax')
])

lstm_model.compile(optimizer='rmsprop', loss='mse')

In [6]:
# hacemos el FIT
history = lstm_model.fit(X_train, y_train, epochs=5, validation_split=0.2)

Train on 9553 samples, validate on 2389 samples
Epoch 1/5
9553/9553 [==============================] - 39s 4ms/step - loss: 0.0249 - val_loss: 0.0246
Epoch 2/5
9553/9553 [==============================] - 38s 4ms/step - loss: 0.0244 - val_loss: 0.0245
Epoch 3/5
9553/9553 [==============================] - 38s 4ms/step - loss: 0.0243 - val_loss: 0.0244
Epoch 4/5
9553/9553 [==============================] - 38s 4ms/step - loss: 0.0242 - val_loss: 0.0243
Epoch 5/5
9553/9553 [==============================] - 38s 4ms/step - loss: 0.0241 - val_loss: 0.0242


In [7]:
# Si queremos predecir un el siguiente caracter de un texto, nos devolverá como predicción un array de 37. Cada posición representa un caracter.
# Obtenemos el índice con el valor más alto y ese será nuestro caracter predecido.